In [ ]:
from ultralytics import YOLO
import cv2
import json

model = YOLO("../best.pt")  

video_path = "../test_video.mp4" 
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

output_json = []

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, save=False)

    frame_data = {
        "frame_id": frame_idx,
        "timestamp": frame_idx / fps, 
        "detections": []
    }
    
    for result in results:
        json_result = json.loads(result.tojson())
        frame_data["detections"].extend(json_result)

    output_json.append(frame_data)
    frame_idx += 1

    print(f"Processed frame {frame_idx}/{frame_count}")

cap.release()

with open("video_detection_results.json", "w", encoding="utf-8") as f:
    json.dump(output_json, f, indent=4, ensure_ascii=False)

print("영상 검출 결과가 'video_detection_results.json' 파일로 저장되었습니다.")

In [3]:
import json
import numpy as np
from statistics import mode, StatisticsError
import matplotlib.pyplot as plt
from collections import Counter
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

plt.rcParams['font.family'] = 'Malgun Gothic'  
plt.rcParams['axes.unicode_minus'] = False  

def calculate_statistics(num):
    counter = Counter(num)  
    total_count = len(num) 
    try:
        mod = mode(num)     
    except StatisticsError:
        mod = "없음 (동일 빈도)"
    
    class_counts = dict(counter)
    class_ratios = {k: v / total_count * 100 for k, v in class_counts.items()}  

    stats = {
        "최빈값": mod,
        "클래스별 빈도수": class_counts,
        "클래스별 비율(%)": class_ratios,
        "총 데이터 개수": total_count
    }
    return stats

with open("../video_detection_results.json", "r", encoding="utf-8") as f:
    data = json.load(f)

classValue = []
for frame in data:
    for detection in frame["detections"]:
        classValue.append(detection["class"])

stats = calculate_statistics(classValue)

root = tk.Tk()
root.title("클래스 통계 시각화")

fig = plt.Figure(figsize=(12, 6))

ax1 = fig.add_subplot(1, 2, 1)
classes = list(stats["클래스별 빈도수"].keys())
counts = list(stats["클래스별 빈도수"].values())
ax1.bar(classes, counts, color='skyblue', edgecolor='black')
ax1.set_title('클래스별 빈도수 (막대 그래프)')
ax1.set_xlabel('Class 값')
ax1.set_ylabel('빈도')
ax1.set_xticks(classes)

ax2 = fig.add_subplot(1, 2, 2)
ratios = list(stats["클래스별 비율(%)"].values())
labels = [f"Class {k}" for k in stats["클래스별 비율(%)"].keys()]
ax2.pie(ratios, labels=labels, autopct='%1.1f%%', startangle=90, colors=plt.cm.Pastel1.colors)
ax2.set_title('클래스별 비율 (파이 차트)')
ax2.axis('equal')

stats_text = (
    f"최빈값: {stats['최빈값']}\n"
    f"총 데이터 개수: {stats['총 데이터 개수']}\n\n"
    "클래스별 빈도수:\n" + "\n".join([f"Class {k}: {v}" for k, v in stats["클래스별 빈도수"].items()]) + "\n\n"
    "클래스별 비율(%):\n" + "\n".join([f"Class {k}: {v:.1f}%" for k, v in stats["클래스별 비율(%)"].items()])
)
fig.text(0.91, 0.5, stats_text, ha="left", va="center", fontsize=10, bbox={"facecolor": "white", "alpha": 0.5, "pad": 5})

canvas = FigureCanvasTkAgg(fig, master=root)
canvas.draw()
canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

root.mainloop()

FileNotFoundError: [Errno 2] No such file or directory: '../video_detection_results.json'

In [1]:
# 확인일시 이름, 출석 여부 json 출력

def st_name(mod):
    if mod == 1: name = "박현호"
    elif mod == 0: name = "권겸"
    elif mod == 2: name = "노시태"
    return name
real_Master = input("자리 주인의 이름 입력: ")

if st_name(mod) == real_Master: print("출석")
else: print("결석")

KeyboardInterrupt: Interrupted by user